# Jupyter Example
## LAMMPS: flow/in.flow.pois
Reference: https://github.com/lammps/lammps/blob/master/examples/flow/in.flow.pois  
Required files: None  
Description: 2-d LJ Poiseuille flow simulation

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()
L.dimension("2")
L.boundary("p s p")
L.atom_style("atomic")
L.neighbor("0.3 bin")
L.neigh_modify("delay 5")

# create geometry
L.lattice("hex 0.7")
L.region("box block 0 20 0 10 -0.25 0.25")
L.create_box("3 box")
L.create_atoms("1 box")
L.mass("1 1.0")
L.mass("2 1.0")
L.mass("3 1.0")

# LJ potentials
L.pair_style("lj/cut 1.12246")
L.pair_coeff("* * 1.0 1.0 1.12246")

# define groups
L.region("1 block INF INF INF 1.25 INF INF")
L.group("lower region 1")
L.region("2 block INF INF 8.75 INF INF INF")
L.group("upper region 2")
L.group("boundary union lower upper")
L.group("flow subtract all boundary")
L.set("group lower type 2")
L.set("group upper type 3")

LAMMPS output is captured by PyLammps wrapper


['Setting atom values ...', '  60 settings made for type']

In [3]:
# initial velocities
L.compute("mobile flow temp")
L.velocity("flow create 1.0 482748 temp mobile")
L.fix("1 all nve")
L.fix("2 flow temp/rescale 200 1.0 1.0 0.02 1.0")
L.fix_modify("2 temp mobile")

# Poiseuille flow
L.velocity("boundary set 0.0 0.0 0.0")
L.fix("3 lower setforce 0.0 0.0 0.0")
L.fix("4 upper setforce 0.0 NULL 0.0")
L.fix("5 upper aveforce 0.0 -1.0 0.0")
L.fix("6 flow addforce 0.5 0.0 0.0")
L.fix("7 all enforce2d")

In [4]:
# output
L.dump("1 all atom 500 dump.flow")

#L.dump("2 all image 100 image.*.jpg type type zoom 1.6 adiam 1")
#L.dump_modify("2 pad 5")    

L.dump("3 all movie 100 mov.mp4 type type zoom 1.6 adiam 1")
L.dump_modify("3 pad 5")

In [5]:
# Run
L.thermo("500")
L.thermo_modify("temp mobile")
L.timestep("0.003")
L.run("10000")

['Neighbor list info ...',
 '  update every 1 steps, delay 5 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 1.42246',
 '  ghost atom cutoff = 1.42246',
 '  binsize = 0.71123, bins = 37 32 1',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair lj/cut, perpetual',
 '      attributes: half, newton on',
 '      pair build: half/bin/atomonly/newton',
 '      stencil: half/bin/2d/newton',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 0.003',
 'Per MPI rank memory allocation (min/avg/max) = 4.32 | 4.32 | 4.32 Mbytes',
 'Step Temp E_pair E_mol TotEng Press Volume ',
 '       0            1            0            0   0.71190476   0.52314537    571.54286 ',
 '     500    1.0990751  -0.36396768            0   0.41846911     2.522517    575.55806 ',
 '    1000            1  -0.34609791            0   0.36580685    1.9736175    584.67179 ',
 '   

In [6]:
L.undump(3)
L.video("mov.mp4")